# Classe DatasetBase

Essa classe é responsável por criar o objeto que vai conter as informações dos Datasets criados.

## Métodos:
* __init__: Construtor da classe onde são passados os seguintes parâmetros
  * bucket_name_input: Nome do bucket onde está o dataset.
  * path_input: Caminho do arquivo no bucket.
  * delimiter_input: Delimitador utilizado no arquivo.
  * extension: Extensão utilizada no arquivo.
  
  
* **loadDatabase**: Método para criar o bucket, ler o arquivo e retornar o Dataframe referente. Utiliza os seguintes parâmetros:
  * bucket_name: Nome do bucket onde está o dataset.
  * complete_path: Caminho completo do arquivo no sistema de arquivos do Databricks.
  * extension: Extensão utilizada no arquivo.
  * delimiter: Delimitador utilizado no arquivo.
  
* **writeDataset**: Método para criar o bucket e escrever um Dataframe em formato parquet no bucket informado. Utiliza os seguintes parâmetros:
  * df: Dataframe que será utilizado para escrever no caminho informado.
  * bucket_name_output: Nome do bucket onde será escrito o dataset.
  * complete_path_output: Caminho completo onde arquivo será escrito no sistema de arquivos do Databricks.
  
* **createBucket**: Método que cria o bucket AWS no sistema de arquivos do Databricks. Utiliza os seguintes parâmetros:
  * bucket_name: Nome do bucket que será carregado.
  
* **deleteBucket**: Método que deleta o bucket AWS do sistema de arquivos do Databricks. Utiliza os seguintes parâmetros:
  * bucket_name: Nome do bucket que será deletado.

In [2]:
class DatasetBase:
    def __init__(self, bucket_name_input, path_input, delimiter_input, extension):
        self.bucket_name_input = bucket_name_input
        self.path_input = path_input
        self.delimiter_input = delimiter_input
        self.extension = extension
     
    @staticmethod
    def loadDatabase(bucket_name, complete_path, extension='parquet', delimiter=','):
        DatasetBase.createBucket(bucket_name)
        
        if extension == 'csv':
            df = spark.read.options(header=True,delimiter=delimiter).csv(complete_path)
        elif extension == 'json':
            df = spark.read.json(complete_path)
        elif extension == 'parquet':
            df = spark.read.parquet(complete_path)
        else:
            raise 'ERROR: Formato não compatível'
            
        return df
    
    @staticmethod
    def writeDataset(df, bucket_name_output, complete_path_output):
        DatasetBase.createBucket(bucket_name_output)
        df.write.format('parquet').mode("overwrite").save(complete_path_output)
        return df
      
    @staticmethod
    def createBucket(bucket_name):
        access_key = dbutils.secrets.get(scope = "aws", key = "aws-access-key")
        secret_key = dbutils.secrets.get(scope = "aws", key = "aws-secret-key").replace("/", "%2F")
        encription = "sse-s3"
        
        if not any(mount.mountPoint == f'/mnt/{bucket_name}' for mount in dbutils.fs.mounts()):
            dbutils.fs.mount(f's3a://{access_key}:{secret_key}@{bucket_name}', f'/mnt/{bucket_name}', encription)            
        return dbutils.fs.mounts()
    
    @staticmethod
    def deleteBucket(bucket_name):
        if any(mount.mountPoint == f'/mnt/{bucket_name}' for mount in dbutils.fs.mounts()):
            dbutils.fs.unmount(f'/mnt/{bucket_name}')        
        return dbutils.fs.mounts()